In [1]:
!pip install selenium

# Mission to Mars Scraping

This is the Web Scraping Homework for the Trilogy/USC Data Science BootCamp

## Part 1- Nasa Mars News

* Scrape the NASA Mars News Site(https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [2]:
#import libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import pymongo

In [6]:
# declaring element names to grab
article_header = 'slide'
content_title = 'content_title'
content_body = 'article_teaser_body'
wait_element = 'news'

#
driver = webdriver.Chrome('/Users/arazohanessian/Desktop/web-scraping-challenge/chromedriver')

# get url and wait until page is fully loaded before proceeding
driver.get('https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest')
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID,wait_element))
    )
except:
    pass

In [7]:
#return a list of divs that hold the articles
articles = driver.find_elements_by_class_name(article_header)


In [8]:
print(articles[0].find_element_by_class_name(content_title).text)
print(articles[0].find_element_by_class_name(content_body).text)

How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus
Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.


In [ ]:
for article in articles:
    try:
        print(article.find_element_by_class_name(content_body).text) 
    except:
        pass

# Part 2 JPL Mars Space Images

In [9]:
import requests
from bs4 import BeautifulSoup

In [10]:
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
featured_image_response = requests.get(featured_image_url)

if featured_image_response.status_code ==200:
    featured_image_html = featured_image_response.text

In [11]:
featured_image_soup = BeautifulSoup(featured_image_html, 'html.parser')
image_element = featured_image_soup.find("a", {"class": "button fancybox"}).get('data-fancybox-href')
"https://www.jpl.nasa.gov/" + image_element

'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA19041_ip.jpg'

# Part 3 Mars Weather

In [12]:
weather_tweets_url = "https://twitter.com/marswxreport?lang=en"
driver.get(weather_tweets_url)

In [13]:
#get using Xpath
mars_weather = driver.find_element_by_xpath('/html/body/div/div/div/div[2]/main/div/div/div/div/div/div/div/div/div[2]/section/div/div/div[1]/div/div/div/article/div/div[2]/div[2]/div[2]/div[1]/div/span')
mars_weather.text

'InSight sol 500 (2020-04-22) low -93.8ºC (-136.8ºF) high -3.5ºC (25.6ºF)\nwinds from the WNW at 4.5 m/s (10.1 mph) gusting to 20.4 m/s (45.6 mph)\npressure at 6.70 hPa'

# Part 4 Mars Facts

In [14]:
import pandas as pd


In [15]:
mars_fact_url = "https://space-facts.com/mars/"
all_mars_fact_tables = pd.read_html(mars_fact_url)
df_mars_facts = all_mars_fact_tables[0]
df_mars_facts.columns = ["Measure", "Number"]
df_mars_facts.head(10)

,Measure,Number
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


## Mars Hemispheres

In [16]:
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
mars_hemisphere_response = requests.get(mars_hemisphere_url)

if mars_hemisphere_response.status_code == 200:
    mars_hemisphere_html = mars_hemisphere_response.text

In [31]:
#creating a function that will apply to each link: visit each link and extract title link
def get_full_image_link(link):
    response = requests.get(link)
    
    if response.status_code == 200:
        html = response.text
        
    soup = BeautifulSoup(html, "html.parser")
    
    img_link = soup.find("a", text="Original").get("href")
    title = soup.find("h2", class_="title").text.split(" Enhanced")[0]
    
    return [title, img_link]
    

In [32]:
mars_hemisphere_soup = BeautifulSoup(mars_hemisphere_html, 'html.parser')
link_elements = mars_hemisphere_soup.findAll("a", {"class": "itemLink product-item"})
image_links = [dict(title=get_full_image_link("https://astrogeology.usgs.gov/" + le.get('href'))[0], img_url=get_full_image_link("https://astrogeology.usgs.gov/" + le.get('href'))[1]) for le in link_elements]
image_links

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}]